In [ ]:
#Import python libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
#ignore warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("df_merged.csv")

In [ ]:
pg_lat = []
pg_long = []

def lat_long_to_PG_lat_long(x, list_lat_long):
    for i in range(len(x)):
        if x[i] >= 0:
            if x[i] % 1 >= 0 and x[i] % 1 <= 0.5:
                list_lat_long.append(int(x[i]) + 0.25)
            else:
                list_lat_long.append(int(x[i]) + 0.75)

        if x[i] < 0:
            if x[i] % 1 >= 0 and x[i] % 1 <= 0.5:
                list_lat_long.append(int(x[i]) - 0.75)
            else:
                list_lat_long.append(int(x[i]) - 0.25)


lat_long_to_PG_lat_long(df["lat"], pg_lat)
lat_long_to_PG_lat_long(df["lng"], pg_long)

df['pg_lat'] = pg_lat
df['pg_long'] = pg_long

In [ ]:
gid = pd.read_csv("../PRIO GRID spine.csv")

In [ ]:
a = df.merge(gid, how="left", left_on=[
             "pg_lat", "pg_long"], right_on=["lat", "lon"])
a['gid'] = a['gid'].astype('Int64')

In [ ]:
#Fill missing values in capex with those of same row in capital investment
a['capital investment'] = a['capital investment'].fillna(a['capex'])
a_modified = a[['project date','investing company','parent company', 'source country',
       'source state', 'source city', 'destination country',
       'destination state', 'admin region', 'destination city',
       'lat_x', 'lng', 'pg_lat', 'pg_long', 'gid', 'industry sector',
       'subsector', 'cluster', 'industry activity', 'capital investment',
       'estimated', 'jobs created', 'estimated 1', 'project type',
       'motive', 'email', 'contact name 1 ', 'position 1 ', 'email 1 ',
       'telephone 1 ', 'contact name 2 ', 'position 2 ', 'email 2 ',
       'telephone 2 ', 'contact name 3 ', 'position 3 ', 'email 3 ',
       'telephone 3 ']]
a_modified.rename(columns={'gid':'PRIO GRID ID','motive':'motive description','lat_x':'dest_latitude','lng':'dest_longitude'}, inplace=True)

In [ ]:
# a_modified.to_csv("all_combined_with_prio_grid_id.csv", index=False)

MONTHLY

In [ ]:
#Rename project date to project_date
a_modified['Project No.'] = 1
a_modified.rename(columns={'project date':'project_date'}, inplace=True)
a_modified['project_date'] = pd.to_datetime(a_modified['project_date'], format='%Y-%m-%d')
a_modified['project_date_monthly'] = a_modified.project_date.dt.to_period("M")

In [ ]:
grouped_data = a_modified.groupby(['project_date_monthly','PRIO GRID ID']).sum()
grouped_data.reset_index(inplace=True, drop=False)


In [ ]:
grouped_data = grouped_data[['PRIO GRID ID','project_date_monthly','Project No.','capital investment']]
grouped_data.rename(columns={'Project No.':'Number of FDI projects','project_date_monthly':'Month-year'}, inplace=True)
grouped_data.reset_index(inplace=True, drop=True)

In [ ]:
grouped_data

Yearly

In [ ]:
a_modified['project_date_yearly'] = a_modified.project_date.dt.to_period("Y")

In [ ]:
grouped_data_year = a_modified.groupby(['project_date_yearly','PRIO GRID ID']).sum()
grouped_data_year.reset_index(inplace=True, drop=False)
grouped_data_year = grouped_data_year[['PRIO GRID ID','project_date_yearly','Project No.','capital investment']]
grouped_data_year.rename(columns={'Project No.':'Number of FDI projects','project_date_yearly':'Year'}, inplace=True)
grouped_data_year.reset_index(inplace=True, drop=True)

In [ ]:
grouped_data_year

Export Data

In [ ]:
# grouped_data.to_csv("monthly_fdi_projects.csv", index=False)
# grouped_data_year.to_csv("yearly_fdi_projects.csv", index=False)

Add COW – correlates of war codes

In [ ]:
country_names_gid = pd.read_csv('prio grid full panel with country names.csv')

In [ ]:
country_names_gid.sort_values(by=['statenme'], inplace=True)
country_names_gid.reset_index(inplace=True, drop=True)

In [ ]:
a_modified['project_date_yearly'] = a_modified.project_date.dt.to_period("Y")
grouped_data_year = a_modified.groupby(['project_date_yearly','PRIO GRID ID']).sum()
grouped_data_year.reset_index(inplace=True, drop=False)
# grouped_data_year = grouped_data_year[['PRIO GRID ID','project_date_yearly','Project No.','capital investment']]
grouped_data_year.rename(columns={'Project No.':'Number of FDI projects','project_date_yearly':'Year'}, inplace=True)
grouped_data_year.reset_index(inplace=True, drop=True)

In [ ]:
#Join the two dataframes grouped_data_year and country_names_gid on PRIO GRID ID
grouped_data_year_with_COW = grouped_data_year.merge(country_names_gid, how="left", left_on="PRIO GRID ID", right_on="gid")

In [ ]:
grouped_data_year_with_COW = grouped_data_year_with_COW.groupby(['Year','statenme']).sum()[['capital investment','jobs created','Number of FDI projects']]
grouped_data_year_with_COW.reset_index(inplace=True, drop=False)
grouped_data_year_with_COW

In [ ]:
#Create a dataframe with the unique country names and each country name will have corresponding another column with years from 1990 to 2022
country_names_gid_new = pd.DataFrame()
country_names_gid_new['Year'] = pd.date_range(start='2003', end='2020', freq='Y').year
# Join every row of country_names_gid with every row of country_names_gid_new
country_names_gid_new = country_names_gid_new.assign(key=1).merge(pd.DataFrame(country_names_gid['statenme'].unique()).assign(key=1), on='key').drop('key', 1)

In [ ]:
grouped_data_year_with_COW['Year'] = grouped_data_year_with_COW['Year'].astype('int64')
#join the two dataframes grouped_data_year_with_COW and country_names_gid_new on country name and year
grouped_data_year_with_COW_new = grouped_data_year_with_COW.merge(country_names_gid_new, how="right", left_on=["statenme","Year"], right_on=[0,"Year"])